In [111]:
import airfoil_simulator
import tensorflow as tf
import numpy as np
import lstm_model
import time

In [68]:
airfoil_simulator.objective([-1,-1])

-1

In [134]:
def obj_airfoil_lift_drag(x):
    x = np.array(x)
    obj_value = -1*airfoil_simulator.objective(x.reshape(-1))
    return np.array(obj_value).reshape(1,1)

In [135]:
obj_airfoil_lift_drag([-0.2,-0.1])

array([[-35.17391304]])

In [136]:
# imp.reload(lstm_model) # remove this when it's ready

best_model_2d='rnn-cell-2d-1497731697'
steps= 30
dim = 2
x_0 = [-0.1,-0.1]

print("Starting at %s and optimize for %d steps" % (starting_point, steps) )

starting_point = np.array(x_0).reshape(1,dim)
sess, model_params = lstm_model.load_trained_model(best_model_2d)
start_time = time.time()
sample_x, sample_y = lstm_model.generate_sample_sequence(sess, model_params, starting_point, steps = steps, \
    obj_func=obj_airfoil_lift_drag \
)

sample_y = np.array(sample_y).flatten()
min_y_idx = np.argmin(sample_y)

print("Min -L/D found %f at %s" % (sample_y[min_y_idx], sample_x[min_y_idx].flatten()) )
print("Took %f mins" % (( time.time() - start_time ) / 60.0) )

Starting at [[-0.1 -0.1]] and optimize for 30 steps
./trained_models/rnn-cell-2d-1497731697
Load rnn-cell-2d-1497731697 model
+---------------------+-----------------------------------------------+
| Parameter           | Value                                         |
+---------------------+-----------------------------------------------+
| dim                 | 2                                             |
| epochs              | 600                                           |
| forget_bias         | 5.0                                           |
| gp_length           | 0.4714045207910317                            |
| learning_rate_final | 0.0001                                        |
| learning_rate_init  | 0.005                                         |
| loss_function       | MIN                                           |
| model_path          | ./trained_models/rnn-cell-2d-1497731697/model |
| n_bumps             | 6                                             |
| n_hidden

In [137]:
np.array(sample_x).flatten().reshape(-1,2)

array([[-0.1       , -0.1       ],
       [-0.04056527, -0.50768769],
       [-0.06366195, -0.72018832],
       [-0.11230899, -0.81141412],
       [-0.10782368, -0.84135419],
       [-0.10101961, -0.84858823],
       [-0.09720795, -0.86200374],
       [ 0.74546462, -0.72179091],
       [ 0.7557084 , -0.5779956 ],
       [ 0.7676158 , -0.30789408],
       [ 0.73047608, -0.08267315],
       [ 0.6079188 ,  0.05495527],
       [ 0.28563285,  0.10342833],
       [-0.19332938,  0.10043699],
       [-0.35942483,  0.28277338],
       [-0.39660794,  0.25478691],
       [-0.41899604,  0.33561343],
       [-0.43076259,  0.39048189],
       [-0.43305415,  0.40295106],
       [-0.43330002,  0.40560043],
       [-0.43330127,  0.40629178],
       [-0.43328434,  0.40650254],
       [-0.43327171,  0.40656972],
       [-0.43326241,  0.40658808],
       [-0.43325448,  0.40658826],
       [-0.43324733,  0.406582  ],
       [-0.43324012,  0.40657371],
       [-0.4332332 ,  0.40656459],
       [-0.43322635,

In [138]:
sample_y

array([-40.30785563,  -5.28510116, -12.84402049, -10.98022835,
       -10.62955516, -10.53932021,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        , -79.55147808, -81.67213115, -73.39037433,
       -81.83564568, -87.23698193, -88.96478122, -89.15778252,
       -89.23240938, -89.26439232, -89.2750533 , -89.2750533 ,
       -89.2750533 , -89.2750533 , -89.2750533 , -89.2750533 ,
       -89.2750533 , -89.2750533 , -89.2750533 ])